In [ ]:
import torch
import time
import numpy as np

from ase import Atoms
from ase.visualize import view
from ase.build import bulk
from ase.units import GPa
from ase.spacegroup import crystal

from mattersim.applications.batch_relax import BatchRelaxer
from mattersim.forcefield.potential import Potential
from mattersim.datasets.utils.build import build_dataloader
from mattersim.forcefield.potential import MatterSimCalculator
from mattersim.applications.relax import Relaxer
from mattersim.applications.moldyn import MolecularDynamics

from utils.visualisation import plot_potential, plot_relaxation, visualise_structure

Molecular Dynamics simulation only accept ase atoms with an attached calculator.

*Args:*
- atoms (Union[Atoms, Structure]): ASE atoms object contains structure information and calculator.
- ensemble (str, optional): Simulation ensemble choosen. Defaults to nvt_nose_hoover'
- temperature (float, optional): Simulation temperature, in Kelvin. Defaults to 300 K.
- timestep (float, optional): The simulation time step, in fs. Defaults to 1 fs.
- taut (float, optional): Characteristic timescale of the thermostat, in fs. If is None, automatically set it to 1000 * timestep.
- trajectory (Union[str, Trajectory], optional): Attach trajectory object. If trajectory is a string a Trajectory will be constructed. Defaults to None, which means for no trajectory.
- logfile (str, optional): If logfile is a string, a file with that name will be opened. Defaults to '-', which means output to stdout.
- loginterval (int, optional): Only write a log line for every loginterval time steps. Defaults to 10.
- append_trajectory (bool, optional): If False the trajectory file to be overwriten each time the dynamics is restarted from scratch. If True, the new structures are appended to the trajectory file instead.


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"MatterSim running on {device}")

MatterSim running on cuda


In [4]:
model = 5

In [51]:
# Define the L1₀ unit cell parameters
a = 3.85  # Lattice constant in x and y
c = 3.72  # Lattice constant in z (slightly compressed)

# Define the atomic positions and symbols
positions = [
    (0, 0, 0),               # Fe atom
    (0.5 * a, 0.5 * a, 0.5 * c),  # Pt atom
]

symbols = ['Fe', 'Pt']

# Create the unit cell for L1₀ FePt
structure = Atoms(symbols="FePt",
                 positions=positions,
                 cell=[(a, 0, 0), (0, a, 0), (0, 0, c)],
                 pbc=True)  # Periodic boundary conditions

In [52]:
structure.positions

array([[0.   , 0.   , 0.   ],
       [1.925, 1.925, 1.86 ]])

In [53]:
structure.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)

/usr/local/lib/python3.12/dist-packages/mattersim/forcefield/potential.py:896: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, map_location=

In [54]:
print(f"Energy (eV)                 = {structure.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {structure.get_potential_energy()/len(structure)}")
print(f"Forces of first atom (eV/A) = {structure.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {structure.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {structure.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.912310600280762
Energy per atom (eV/atom)   = -5.456155300140381
Forces of first atom (eV/A) = [ 1.2293458e-06  1.2218952e-06 -1.3187528e-06]
Stress[0][0] (x-x) (eV/A^3) = 0.1328575326803123
Stress[0][0] (x-x) (GPa)    = 21.286123275756832


In [55]:
structure.cell.cellpar()

array([ 3.85,  3.85,  3.72, 90.  , 90.  , 90.  ])

In [42]:
trajectory_file = "md_trajectory.traj"
log_file = "md_simulation.log"

In [ ]:
md = MolecularDynamics(
    atoms=structure,
    ensemble="nvt_nose_hoover",
    temperature=300,
    timestep=1.0,
    loginterval=10,
    logfile = log_file,
    trajectory=trajectory_file  # Save trajectory
)

In [57]:
md.run(n_steps=1000)

In [58]:
md.atoms.cell.cellpar()

array([ 3.85,  3.85,  3.72, 90.  , 90.  , 90.  ])

In [59]:
md.atoms.get_positions()

array([[-2.75081276, -0.49308007,  0.03647099],
       [ 2.71245126,  2.0661497 ,  1.84955971]])

In [60]:
print(f"Energy (eV)                 = {md.atoms.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {md.atoms.get_potential_energy()/len(structure)}")
print(f"Forces of first atom (eV/A) = {md.atoms.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {md.atoms.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {md.atoms.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -11.364585876464844
Energy per atom (eV/atom)   = -5.682292938232422
Forces of first atom (eV/A) = [ 0.5268571  -0.8831482   0.06431783]
Stress[0][0] (x-x) (eV/A^3) = 0.13525236037962535
Stress[0][0] (x-x) (GPa)    = 21.669816970825195


In [61]:
_ = visualise_structure(md.atoms)

Structure positions: [[-2.75081276 -0.49308007  0.03647099]
 [ 2.71245126  2.0661497   1.84955971]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.